### Add description of the dataset
for instance a way to download it from kaggle

In [1]:
import re

from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [5]:
data_df = pd.read_csv('data/train.csv', index_col='Id')
print(data_df.columns)
data_df.head()

Index(['Title', 'Body', 'Tags', 'CreationDate', 'Y'], dtype='object')


,Title,Body,Tags,CreationDate,Y
Id,,,,,
34552656,Java: Repeat Task Every Random Seconds,<p>I'm already familiar with repeating tasks e...,<java><repeat>,2016-01-01 00:21:59,LQ_CLOSE
34553034,Why are Java Optionals immutable?,<p>I'd like to understand why Java 8 Optionals...,<java><optional>,2016-01-01 02:03:20,HQ
34553174,Text Overlay Image with Darkened Opacity React...,<p>I am attempting to overlay a title over an ...,<javascript><image><overlay><react-native><opa...,2016-01-01 02:48:24,HQ
34553318,Why ternary operator in swift is so picky?,"<p>The question is very simple, but I just cou...",<swift><operators><whitespace><ternary-operato...,2016-01-01 03:30:17,HQ
34553755,hide/show fab with scale animation,<p>I'm using custom floatingactionmenu. I need...,<android><material-design><floating-action-but...,2016-01-01 05:21:48,HQ


In [9]:
print(data_df.iloc[0])
data_df['Body'].iloc[0]

Title                      Java: Repeat Task Every Random Seconds
Body            <p>I'm already familiar with repeating tasks e...
Tags                                               <java><repeat>
CreationDate                                  2016-01-01 00:21:59
Y                                                        LQ_CLOSE
Name: 34552656, dtype: object


'<p>I\'m already familiar with repeating tasks every n seconds by using Java.util.Timer and Java.util.TimerTask. But lets say I want to print "Hello World" to the console every random seconds from 1-5. Unfortunately I\'m in a bit of a rush and don\'t have any code to show so far. Any help would be apriciated.  </p>\n'

In [ ]:
def remove_code_blocks(index: int, text: str) -> str:
    """
    Removes code blocks from a string of HTML text.
    The function searches for code blocks by looking
    for HTML tags <pre and </pre>, and returns the
    remaining text without these code blocks.

    INPUT
        index: Index of the text to be processed.
        text: The HTML text to be processed.

    OUTPUT
        processed_text: The processed text with code blocks removed.
    """

    code_block_present = False
    
    start_tags, end_tags = [], []
    
    for i in range(len(text)):
    
        if text[i:i+4] == '<pre':
    
            start_tags.append(i)
            code_block_present = True
    
        if text[i:i+6] == '</pre>':
    
            end_tags.append(i)
    
    if not code_block_present:
    
        return text
    
    processed_text = ''
    
    for j in range(len(start_tags)):
    
        if j == 0:
    
            processed_text += text[:start_tags[j]-1]
    
        elif j != 0 and j < len(start_tags)-1:
    
            processed_text += text[end_tags[j-1]+7:start_tags[j]-1]
    
    if len(end_tags) == len(start_tags):
    
        processed_text += text[end_tags[j]+7:len(text)-1]
    
    return processed_text


In [ ]:
# Create a new empty dataframe with the desired columns
new_dataframe = pd.DataFrame(columns=['Id', 'Title', 'Body', 'Tags', 'CreationDate', 'Y'])

# Iterate over the rows of the original dataset
for index, row in original_dataframe.iterrows():

    # Skip specific rows by index
    if index in [8166, 8778, 24599, 32425]:
        continue

    # Process the body text
    body_text = re.sub('<a.*?/a>', '', row['Body']) # Remove <a> tags
    body_text = remove_code_blocks(index, body_text) # Remove code blocks
    body_text = re.sub('<code>.*?</code>', '', body_text) # Remove inline code
    body_text = BeautifulSoup(body_text, 'html.parser').get_text() # Remove other HTML tags
    body_text = body_text.replace('\n', ' ').lower() # Remove newlines and convert to lowercase
    body_text = body_text.replace('-', ' ').replace('\'', '').replace('\"', ' ').replace(':', ' ') # Remove certain characters
    body_text = body_text.replace('(', ' ').replace(')', ' ').replace('.', ' ').replace(',', ' ').replace('?', ' ').replace('!', ' ') # Remove certain characters
    body_text = re.sub('\s+', ' ', body_text) # Replace multiple spaces with a single space

    # Process the tags
    tags = re.search('<(.*)>', row['Tags']).group(1).replace('><', ' ') # Extract the tags and remove <>
    
    # Create a new dictionary with the processed data
    new_row = {
        'Id': row['Id'], 
        'Title': row['Title'], 
        'Body': body_text, 
        'Tags': tags, 
        'CreationDate': row['CreationDate'], 
        'Y': row['Y']
    }
    
    # Add the dictionary to the new dataframe
    new_dataframe = new_dataframe.append(new_row, ignore_index=True)
